In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Updating the dataset using Snorkel + LogisticRegression

In [ ]:

import re
import json
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from snorkel.labeling import LabelingFunction, PandasLFApplier
from snorkel.labeling.model import LabelModel
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

train_data = load_json("/kaggle/input/pumaaa/puma_dataset/train.json")
valid_data = load_json("/kaggle/input/pumaaa/puma_dataset/valid.json")
test_data = load_json("/kaggle/input/pumaaa/puma_dataset/test.json")


def create_labeled_df(data):
    rows = []
    for d in data:
        for label, spans in d.get("labelled_answer_spans", {}).items():
            for span in spans:
                rows.append({
                    "uri": d["uri"],
                    "question": d["question"],
                    "text": span["txt"],
                    "label": label
                })
    return pd.DataFrame(rows)

train_df = create_labeled_df(train_data)
valid_df = create_labeled_df(valid_data)

PERSPECTIVES = ["EXPERIENCE", "INFORMATION", "CAUSE", "SUGGESTION", "QUESTION"]
train_df = train_df[train_df["label"].isin(PERSPECTIVES)]
train_df["label_id"] = train_df["label"].apply(lambda x: PERSPECTIVES.index(x))


# Sentence Embedding

embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
X_train = embedding_model.encode(train_df["text"].tolist(), convert_to_numpy=True)
y_train = train_df["label_id"].values


# Train Classifier

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)


# Snorkel Labeling Functions

vectorizer = TfidfVectorizer(max_features=50, stop_words="english")
tfidf_matrix = vectorizer.fit_transform(train_df["text"])
feature_names = vectorizer.get_feature_names_out()

def create_lf(label_idx, keywords):
    pattern = r"\\b(" + "|".join(map(re.escape, keywords)) + r")\\b"
    def f(x): return label_idx if re.search(pattern, x.text, re.IGNORECASE) else -1
    return LabelingFunction(name=f"lf_{PERSPECTIVES[label_idx]}", f=f)

labeling_functions = []
for idx, label in enumerate(PERSPECTIVES):
    sample_texts = train_df[train_df["label"] == label]["text"]
    all_words = " ".join(sample_texts).split()
    keywords = [word for word in all_words if word in feature_names][:10]
    labeling_functions.append(create_lf(idx, keywords))

applier = PandasLFApplier(lfs=labeling_functions)
L_train = applier.apply(df=train_df)

label_model = LabelModel(cardinality=len(PERSPECTIVES), verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100)


# Predict Function

def classify_entry(entry):
    categorized = {label: [] for label in PERSPECTIVES}
    for ans in entry["answers"]:
        sentences = re.split(r"(?<=[.!?])\\s+", ans.strip())
        if not sentences: continue
        L_test = applier.apply(pd.DataFrame({"text": sentences}))
        weak_labels = label_model.predict(L=L_test)
        emb = embedding_model.encode(sentences, convert_to_numpy=True)
        logreg_preds = logreg.predict(emb)
        for i, sent in enumerate(sentences):
            label_idx = weak_labels[i] if weak_labels[i] != -1 else logreg_preds[i]
            perspective = PERSPECTIVES[label_idx]
            categorized[perspective].append(sent)
    return {
        "uri": entry["uri"],
        "spans": categorized
    }


# Apply on Test Data

results = [classify_entry(entry) for entry in test_data]


# Save Result

with open("classified_test_output.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print("Done! Saved results to classified_test_output.json.")


2025-04-13 08:00:44.214919: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744531244.508933      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744531244.598167      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

100%|██████████| 1/1 [00:00<00:00, 1869.12it/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 2668.13it/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Done! Saved results to classified_test_output.json.


In [1]:
!pip install snorkel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing 

In [2]:
pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Training Bart + Pegasus model from above updated dataset with Snorkel + LogisticRegression

In [11]:

import json
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from datasets import load_metric
import evaluate

# Load BART and Pegasus models
bart_model_name = "facebook/bart-large-cnn"
pegasus_model_name = "google/pegasus-xsum"

# Initialize tokenizers and models
bart_tokenizer = AutoTokenizer.from_pretrained(bart_model_name)
bart_model = AutoModelForSeq2SeqLM.from_pretrained(bart_model_name)

pegasus_tokenizer = AutoTokenizer.from_pretrained(pegasus_model_name)
pegasus_model = AutoModelForSeq2SeqLM.from_pretrained(pegasus_model_name)

bart_model = bart_model.to(device)
pegasus_model = pegasus_model.to(device)


# Load evaluation metrics
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

# Function to generate an extractive summary using BART
def generate_extractive_summary(text_list):
    if not text_list:
        return ""
    full_text = " ".join(text_list)
    inputs = bart_tokenizer.encode("summarize: " + full_text, return_tensors="pt", truncation=True, max_length=1024).to(device)
    summary_ids = bart_model.generate(inputs, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Function to refine summary using Pegasus (Abstractive Summarization)
def refine_with_pegasus(summary_text):
    if not summary_text:
        return ""
    inputs = pegasus_tokenizer.encode("summarize: " + summary_text, return_tensors="pt", truncation=True, max_length=512).to(device)
    summary_ids = pegasus_model.generate(inputs, max_length=100, min_length=30, length_penalty=1.8, num_beams=6, early_stopping=True)
    return pegasus_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Load test set with ground-truth summaries and predicted perspectives
with open("/kaggle/input/pumaaa/puma_dataset/test.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

with open("/kaggle/working/classified_test_output.json", "r", encoding="utf-8") as f:
    classified_data = json.load(f)

# Create mapping for reference summaries
summary_lookup = {}
for d in test_data:
    uri = d["uri"]
    ref_summaries = {k.replace("_SUMMARY", ""): v for k, v in d.get("labelled_summaries", {}).items()}
    summary_lookup[uri] = ref_summaries

# Generate summaries
output_data = []
all_preds, all_refs = [], []
all_preds_bert, all_refs_bert = [], []

for entry in classified_data:
    uri = entry["uri"]
    spans = entry["spans"]
    bart_summaries = {cat: generate_extractive_summary(txts) for cat, txts in spans.items()}
    pegasus_summaries = {cat: refine_with_pegasus(txt) for cat, txt in bart_summaries.items()}
    
    # Gather references for BLEU/BERTScore
    ref = summary_lookup.get(uri, {})
    for cat in pegasus_summaries:
        if cat in ref and ref[cat]:
            pred = pegasus_summaries[cat]
            ref_text = ref[cat]
            all_preds.append(pred)
            all_refs.append([ref_text])
            all_preds_bert.append(pred)
            all_refs_bert.append(ref_text)

    output_data.append({
        "uri": uri,
        "spans": spans,
        "extractive_summaries": bart_summaries,
        "refined_summaries": pegasus_summaries
    })

# Compute BLEU and BERTScore
bleu_result = bleu.compute(predictions=all_preds, references=all_refs)
bert_result = bertscore.compute(predictions=all_preds_bert, references=all_refs_bert, lang="en")

# Save results
with open("hybrid_summaries_with_metrics.json", "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2)

# Save metrics
with open("summary_eval_scores.json", "w", encoding="utf-8") as f:
    json.dump({
        "BLEU": bleu_result,
        "BERTScore": {
            "precision": sum(bert_result["precision"]) / len(bert_result["precision"]),
            "recall": sum(bert_result["recall"]) / len(bert_result["recall"]),
            "f1": sum(bert_result["f1"]) / len(bert_result["f1"])
        }
    }, f, indent=2)

print("✅ Summarization complete. Metrics and outputs saved.")


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Summarization complete. Metrics and outputs saved.


### better Classification using Snorkel + SVM + Zero-Shot

In [ ]:

import re
import json
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from snorkel.labeling import LabelingFunction, PandasLFApplier
from snorkel.labeling.model import LabelModel
from transformers import pipeline
from sentence_transformers import SentenceTransformer


# Load JSON-labeled dataset

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

train_data = load_json("/kaggle/input/pumaaa/puma_dataset/train.json")
test_data = load_json("/kaggle/input/pumaaa/puma_dataset/test.json")


# Convert labeled answer spans into DataFrame

def create_labeled_df(data):
    rows = []
    for d in data:
        for label, spans in d.get("labelled_answer_spans", {}).items():
            for span in spans:
                rows.append({
                    "uri": d["uri"],
                    "text": span["txt"],
                    "label": label
                })
    return pd.DataFrame(rows)

labeled_df = create_labeled_df(train_data)


# Define perspective categories

PERSPECTIVES = ["EXPERIENCE", "INFORMATION", "CAUSE", "SUGGESTION", "QUESTION"]


# Sentence Embeddings + SVM Classifier

embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")
print("📌 SentenceTransformer is using device:", embedding_model.device)
X_embeddings = embedding_model.encode(labeled_df["text"].tolist(), convert_to_numpy=True)
svm_classifier = SVC(kernel="linear", probability=True)
svm_classifier.fit(X_embeddings, labeled_df["label"])


# Snorkel Labeling Functions

def create_lf(label, pattern):
    def lf(x):
        text = str(x.text) if hasattr(x, 'text') else str(x)
        return PERSPECTIVES.index(label) if re.search(pattern, text, re.IGNORECASE) else -1
    return LabelingFunction(name=f"lf_{label}", f=lf)

regex_patterns = {}
for label in PERSPECTIVES:
    samples = labeled_df[labeled_df["label"] == label]["text"]
    if not samples.empty:
        common_words = " | ".join(map(re.escape, " ".join(samples).split()[:10]))
        regex_patterns[label] = common_words

labeling_functions = [create_lf(label, regex_patterns[label]) for label in regex_patterns]

applier = PandasLFApplier(lfs=labeling_functions)
L_train = applier.apply(df=labeled_df)

label_model = LabelModel(cardinality=len(PERSPECTIVES), verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100)


# Zero-Shot Classification Fallback

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device="cuda")
print(" Zero-Shot classifier is using device:", classifier.model.device)
def zero_shot_classify(text):
    result = classifier(text, PERSPECTIVES)
    return result["labels"][0]


# Process Unlabeled Data

results = []
for entry in test_data:
    uri = entry["uri"]
    categorized_spans = {perspective: [] for perspective in PERSPECTIVES}
    for ans in entry["answers"]:
        sentences = re.split(r"(?<=[.!?])\s+", ans.strip())
        if not sentences: continue

        L_test = applier.apply(pd.DataFrame({"text": sentences}))
        weak_labels = label_model.predict(L=L_test)
        X_test_embeddings = embedding_model.encode(sentences, convert_to_numpy=True)
        svm_preds = svm_classifier.predict(X_test_embeddings)

        for i, sentence in enumerate(sentences):
            weak_label = weak_labels[i]
            if weak_label != -1:
                final_label = PERSPECTIVES[weak_label]
            else:
                final_label = svm_preds[i] if svm_preds[i] in PERSPECTIVES else zero_shot_classify(sentence)
            categorized_spans[final_label].append(sentence)

    results.append({"uri": uri, "spans": categorized_spans})


# Save Results

with open("classified_perspectives_svm_output.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print(" Classification with SVM + Snorkel + ZSL completed!")


2025-04-13 11:19:31.139209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744543171.366252      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744543171.430130      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 11/11 [00:00<00:00, 9636.04it/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:00<00:00, 7661.83it/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

### Training Bart Model on the above modified dataset with Snorkel + SVM + Zero-Shot

In [12]:

import json
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from datasets import load_metric
import evaluate
from collections import defaultdict

# Load BART model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Load evaluation metrics
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

# Load data
with open("/kaggle/input/pumaaa/puma_dataset/train.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

with open("/kaggle/input/pumaaa/puma_dataset/test.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

with open("/kaggle/working/classified_perspectives_svm_output.json", "r", encoding="utf-8") as f:
    classified_data = json.load(f)

# Create reference summary lookup
ref_lookup = {
    d["uri"]: {k.replace("_SUMMARY", ""): v for k, v in d.get("labelled_summaries", {}).items()}
    for d in train_data + test_data
}
text_lookup = {d["uri"]: d for d in test_data}

# Perspective labels
PERSPECTIVES = ["EXPERIENCE", "INFORMATION", "CAUSE", "SUGGESTION", "QUESTION"]

# Summary length settings
def get_dynamic_lengths(text):
    words = len(text.split())
    min_length = max(30, int(words * 0.2))
    max_length = max(min_length + 10, int(words * 0.5))  # ensure max > min
    max_length = min(200, max_length)  # still cap it
    return max_length, min_length

def generate_extractive_summary(texts):
    if not texts:
        return ""
    full_text = " ".join(texts)
    max_length, min_length = get_dynamic_lengths(full_text)
    inputs = tokenizer("summarize: " + full_text, return_tensors="pt", truncation=True, max_length=1024, padding="max_length").to(device)
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=min_length,
        length_penalty=1.2,
        num_beams=6,
        repetition_penalty=1.1,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Scoring prep
overall_preds, overall_refs = [], []
bert_preds, bert_refs = [], []
perspective_scores = defaultdict(lambda: {"preds": [], "refs": []})
output_data = []

# Summarize and evaluate
for entry in classified_data:
    uri = entry["uri"]
    spans = entry.get("spans", {})
    ref_summaries = ref_lookup.get(uri, {})
    meta = text_lookup.get(uri, {})
    question = meta.get("question", "")
    context = meta.get("context", "")

    summaries = {}
    for perspective, sentences in spans.items():
        summary = generate_extractive_summary(sentences)
        summaries[perspective] = summary

        ref = ref_summaries.get(perspective)
        if ref and summary.strip():
            overall_preds.append(summary)
            overall_refs.append([ref])
            bert_preds.append(summary)
            bert_refs.append(ref)
            perspective_scores[perspective]["preds"].append(summary)
            perspective_scores[perspective]["refs"].append(ref)

    output_data.append({
        "uri": uri,
        "question": question,
        "context": context,
        "spans": spans,
        "summaries": summaries
    })

# Save summaries
with open("final_extractive_summarized_SVM.json", "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2)

# Compute metrics
bleu_score = bleu.compute(predictions=overall_preds, references=overall_refs)
bert_score = bertscore.compute(predictions=bert_preds, references=bert_refs, lang="en")
overall_metrics = {
    "BLEU": bleu_score,
    "BERTScore": {
        "precision": sum(bert_score["precision"]) / len(bert_score["precision"]),
        "recall": sum(bert_score["recall"]) / len(bert_score["recall"]),
        "f1": sum(bert_score["f1"]) / len(bert_score["f1"])
    }
}

# Per-perspective metrics
per_perspective_metrics = {}
for perspective in PERSPECTIVES:
    preds = perspective_scores[perspective]["preds"]
    refs = perspective_scores[perspective]["refs"]
    if preds and refs:
        bleu_score = bleu.compute(predictions=preds, references=[[r] for r in refs])
        bert_score = bertscore.compute(predictions=preds, references=refs, lang="en")
        per_perspective_metrics[perspective] = {
            "BLEU": bleu_score,
            "BERTScore": {
                "precision": sum(bert_score["precision"]) / len(bert_score["precision"]),
                "recall": sum(bert_score["recall"]) / len(bert_score["recall"]),
                "f1": sum(bert_score["f1"]) / len(bert_score["f1"])
            }
        }

# Save metrics
with open("final_extractive_summarized_SVM_metrics.json", "w", encoding="utf-8") as f:
    json.dump({
        "overall_metrics": overall_metrics,
        "per_perspective_metrics": per_perspective_metrics
    }, f, indent=2)

print("✅ Summarization and evaluation complete!")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Summarization and evaluation complete!


### Now Training Bart using few shots labels

In [1]:

import json
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util

# Load BART model for summarization
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

# Load sentence embedding model for similarity search
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Load training data for few-shot learning (built from train.json)
with open("/kaggle/input/pumaaa/puma_dataset/train.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

train_examples = []
for item in train_data:
    summaries = item.get("labelled_summaries", {})
    for k, v in summaries.items():
        train_examples.append((item["question"] + " " + " ".join(item.get("answers", [])), v))

training_texts = [ex[0] for ex in train_examples]
training_summaries = [ex[1] for ex in train_examples]
training_embeddings = embedder.encode(training_texts, convert_to_tensor=True)

# Load test dataset
with open("/kaggle/input/pumaaa/puma_dataset/test.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

# Load SVM-classified perspectives
with open("/kaggle/input/svmmmm/SVM_snorkel_ZSL.json", "r", encoding="utf-8") as f:
    classified_data = json.load(f)

# Create a lookup for test metadata
test_lookup = {entry["uri"]: entry for entry in test_data}

# Few-shot retrieval function
def get_few_shot_examples(test_text, num_examples=3):
    test_embedding = embedder.encode(test_text, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(test_embedding, training_embeddings)[0]
    top_indices = torch.topk(cos_scores, num_examples).indices.tolist()
    return "\n".join([f"Input: {training_texts[i]}\nSummary: {training_summaries[i]}\n" for i in top_indices])

# Generate summary with few-shot context
def generate_extractive_summary(text):
    if not text:
        return ""
    few_shot_prompt = get_few_shot_examples(text)
    prompt = f"Here are examples of good summaries:\n{few_shot_prompt}\nNow summarize:\nInput: {text}\nSummary:"
    input_length = len(text.split())
    max_length = min(250, max(50, input_length // 3))
    min_length = max(25, max_length // 2)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        min_length=min_length,
        length_penalty=1.5,
        num_beams=5,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Process classified perspectives
output_data = []
for entry in classified_data:
    uri = entry["uri"]
    spans = entry["spans"]
    if uri in test_lookup:
        meta = test_lookup[uri]
        question = meta.get("question", "")
        context = meta.get("context", "")
        summaries = {cat: generate_extractive_summary(" ".join(txts)) for cat, txts in spans.items()}
        output_data.append({
            "uri": uri,
            "question": question,
            "context": context,
            "spans": spans,
            "summaries": summaries
        })

# Save output
with open("fewshot_extractive_summarized_output.json", "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2)

print("✅ Extractive summarization with weak supervision completed!")


2025-04-13 20:15:14.999641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744575315.239338      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744575315.299972      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Extractive summarization with weak supervision completed!


In [8]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=02d8d1d482ec7c5402145f622e7cfce51308505e89a546be1105c9074f1097cd
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


### Evaluation Metrices of Bart using few shots labels

In [9]:

import json
# from datasets import load_metric
import evaluate
from collections import defaultdict

# Load the generated summaries
with open("/kaggle/working/fewshot_extractive_summarized_output.json", "r", encoding="utf-8") as f:
    generated_data = json.load(f)

# Load the reference data
with open("/kaggle/input/pumaaa/puma_dataset/test.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

# Build reference lookup
ref_lookup = {
    d["uri"]: {k.replace("_SUMMARY", ""): v for k, v in d.get("labelled_summaries", {}).items()}
    for d in test_data
}

# Initialize evaluation metrics
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")
rouge = evaluate.load("rouge")

# Store overall and per-perspective metrics
overall_preds, overall_refs = [], []
bert_preds, bert_refs = [], []
perspective_scores = defaultdict(lambda: {"preds": [], "refs": []})

PERSPECTIVES = ["EXPERIENCE", "INFORMATION", "CAUSE", "SUGGESTION", "QUESTION"]

for entry in generated_data:
    uri = entry["uri"]
    summaries = entry["summaries"]
    references = ref_lookup.get(uri, {})

    for perspective, generated in summaries.items():
        ref = references.get(perspective)
        if generated and ref:
            overall_preds.append(generated)
            overall_refs.append([ref])
            bert_preds.append(generated)
            bert_refs.append(ref)
            perspective_scores[perspective]["preds"].append(generated)
            perspective_scores[perspective]["refs"].append(ref)

# Compute overall scores
bleu_score = bleu.compute(predictions=overall_preds, references=overall_refs)
bert_score = bertscore.compute(predictions=bert_preds, references=bert_refs, lang="en")
rouge_score = rouge.compute(predictions=bert_preds, references=bert_refs)

overall_metrics = {
    "BLEU": bleu_score,
    "ROUGE": rouge_score,
    "BERTScore": {
        "precision": sum(bert_score["precision"]) / len(bert_score["precision"]),
        "recall": sum(bert_score["recall"]) / len(bert_score["recall"]),
        "f1": sum(bert_score["f1"]) / len(bert_score["f1"])
    }
}

# Compute per-perspective metrics
per_perspective_metrics = {}
for perspective in PERSPECTIVES:
    preds = perspective_scores[perspective]["preds"]
    refs = perspective_scores[perspective]["refs"]
    if preds and refs:
        bleu_score = bleu.compute(predictions=preds, references=[[r] for r in refs])
        bert_score = bertscore.compute(predictions=preds, references=refs, lang="en")
        rouge_score = rouge.compute(predictions=preds, references=refs)
        per_perspective_metrics[perspective] = {
            "BLEU": bleu_score,
            "ROUGE": rouge_score,
            "BERTScore": {
                "precision": sum(bert_score["precision"]) / len(bert_score["precision"]),
                "recall": sum(bert_score["recall"]) / len(bert_score["recall"]),
                "f1": sum(bert_score["f1"]) / len(bert_score["f1"])
            }
        }

# Save metrics
with open("fewshot_summary_metrics.json", "w", encoding="utf-8") as f:
    json.dump({
        "overall_metrics": overall_metrics,
        "per_perspective_metrics": per_perspective_metrics
    }, f, indent=2)

print("✅ Evaluation complete. Metrics saved to fewshot_summary_metrics.json.")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Evaluation complete. Metrics saved to fewshot_summary_metrics.json.
